In [1]:
%run nearest_mrt_station_to_bus_stops.ipynb

['https://ltafarecard.s3.ap-southeast-1.amazonaws.com/202409/origin_destination_bus_202409.zip?X-Amz-Security-Token=IQoJb3JpZ2luX2VjEMr%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaDmFwLXNvdXRoZWFzdC0xIkgwRgIhAKYi%2BKxu%2FuVHMrIzPukbbuS2CTQCOZh7%2FTTJKTBF%2BoEWAiEA59yTtaJCrLXL34Wmz34Vp8pgQzCyOqwihL3B0nJPsP0qwgUIQxAEGgwzNDA2NDUzODEzMDQiDL3iRxzlwU6SvgjLqCqfBavc1KCX8YJix9LvxrV7qdMkHxQkwhj533L5SmJDvwyZ4lS5FRkRdYY8DwR6XIQcjcR%2BkbuhU%2B8n9u4hclJemhd7PYXbgCrf3Dl4ETF%2BeiP1ky5Ct%2FiJzo7VDzN%2BT0owefuIYHDeuUoYQ1BuTJaIsMftL9reMjgac%2BOyLVl8GwLT8xb2of7%2FrwKPxrWHClOplgIKYBBJD4tnZiAcV2HQ%2FeKW7pbAsy3SYpe0ZC4675fCQBZjz9evyyhjf%2F3Roj4Yqr0VZaneMuVV2lSka3jEFsSf%2FGELuJar4jpaVNmOEWrPlLqElHV%2BVyqa4weQYzuqJbRbZ0KQupZHMf3sqz%2BdINfMlj4GlW2IzTAGR71FG%2BIj13o1m4X7bZjAQESoy4rcepH7ldHPLYzR4z%2FfYLauMivOMwoxVjZRLNsZBxYcwBmw9GF%2FOTip6PQcqJzNme4SpffH3%2B1fomnekvxbmADT49cuIIfcXMbMoyIJhdwf0vprZQNrH0cpQD0ngDPiaSbEWAckuBYPxntgVyuXrHe7dkVudEcaB%2BFHaomMASJuWZH6HvIm7hH46tu0ciwats72PPpxTN78eaUOlx5edN%2Bq5MdhB0h3ouN9fwq7ysx0mTUW

In [4]:
%run "MRT Routes visualisation.ipynb"

In [5]:
bus_routes_combined = bus_routes_gdf_3857.groupby(['ServiceNo', 'Direction']).apply(
    lambda x: LineString(x.geometry.tolist())).reset_index()
bus_routes_combined.columns = ['ServiceNo', 'Direction', 'geometry']
bus_routes_combined

C:\Users\kaili\AppData\Local\Temp\ipykernel_21140\3311467830.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  bus_routes_combined = bus_routes_gdf_3857.groupby(['ServiceNo', 'Direction']).apply(


,ServiceNo,Direction,geometry
0,10,1,LINESTRING (11570925.355747303 150749.03122670...
1,10,2,LINESTRING (11551610.561061133 144087.88490860...
2,100,1,LINESTRING (11562943.630025776 150347.12028417...
3,100,2,LINESTRING (11553774.516701924 145960.08849871...
4,100A,1,LINESTRING (11562943.630025776 150347.12028417...
...,...,...,...
719,992,1,LINESTRING (11549515.381030776 150294.50067879...
720,992,2,LINESTRING (11547835.02338783 151709.373424986...
721,993,1,LINESTRING (11548357.891036088 148329.89223418...
722,9A,1,LINESTRING (11569327.609406518 147460.11332459...


In [6]:
bus_routes_combined_4326 = bus_routes_gdf.groupby(['ServiceNo', 'Direction']).apply(
    lambda x: LineString(x.geometry.tolist())).reset_index()
bus_routes_combined_4326.columns = ['ServiceNo', 'Direction', 'geometry']
bus_routes_combined_4326

C:\Users\kaili\AppData\Local\Temp\ipykernel_21140\2815619532.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  bus_routes_combined_4326 = bus_routes_gdf.groupby(['ServiceNo', 'Direction']).apply(


,ServiceNo,Direction,geometry
0,10,1,LINESTRING (103.94339098473914 1.3540755236747...
1,10,2,LINESTRING (103.76988323197696 1.2942534105633...
2,100,1,LINESTRING (103.87168992264617 1.3504661017696...
3,100,2,LINESTRING (103.78932237623975 1.3110673548964...
4,100A,1,LINESTRING (103.87168992264617 1.3504661017696...
...,...,...,...
719,992,1,LINESTRING (103.75106190953444 1.3499935430612...
720,992,2,"LINESTRING (103.735967 1.3627, 103.73349 1.361..."
721,993,1,"LINESTRING (103.740664 1.33235, 103.7388491669..."
722,9A,1,LINESTRING (103.92903818515839 1.3245387431734...


In [7]:
frechet_dist_output = []

for i, row in bus_routes_combined.iterrows():
    line_bus = gpd.GeoSeries([row["geometry"]])
    f_distances = []
    for j, rowj in lines_gdf_3857.iterrows():
        line_mrt = gpd.GeoSeries([rowj["geometry"]])
        dist = line_bus.frechet_distance(line_mrt)
        f_distances.append(dist.iloc[0])
    frechet_dist_output.append({
        "Bus Service No": row["ServiceNo"],
        "Direction": row["Direction"],
        "MRT Line": lines_gdf_3857.loc[f_distances.index(min(f_distances)), "line"], 
        "Frechet Distance" : float(min(f_distances))
    })

In [8]:
fdist = pd.DataFrame(frechet_dist_output).sort_values("Frechet Distance").reset_index().drop("index", axis=1)
fdist

,Bus Service No,Direction,MRT Line,Frechet Distance
0,80,2,NE,2722.779008
1,67,1,DT,2976.784583
2,186,2,CC,3835.798232
3,107,2,NE,3993.444012
4,139,1,CC,4448.334223
...,...,...,...,...
719,193,1,CC,22161.401506
720,192,1,CC,22161.401506
721,248M,1,NS,26217.431775
722,248,1,NS,26217.431775
